# LSTM DeepVaR forecasting

## 1. Import library 

In [38]:
# import required libraries
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler, Normalizer, StandardScaler

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error, root_mean_squared_error

import torch
import torch.nn as nn
import torch.optim as optim

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, clear_output
import calendar
from datetime import datetime
import os
import joblib

## 2.Import dataset

In [44]:
# Read the CSV file
path = "/Users/esther/Desktop/202506final_thesis/VScode/LSTM-wine-streamlit-main/Top37.csv"
df = pd.read_csv(path, delimiter=",")

# Convert DateTime column to date format
df['Datetime'] = pd.to_datetime(df['Datetime'])

# Convert all other columns to numeric format
# First, get all columns except DateTime
numeric_columns = df.columns[1:]

# Convert each column to numeric, coerce errors to NaN
for col in numeric_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Forward fill missing values
df = df.ffill()

# Display the first few rows to verify the changes
print(df.head())

# Check data types
print("\nData types:")
print(df.dtypes)

# Check for any remaining NaN values
print("\nRemaining NaN values:")
print(df.isna().sum())

    Datetime  Leroy Musigny Grand Cru, Cote de Nuits, France  \
0 2015-01-01                                         5218.56   
1 2015-02-01                                         5065.06   
2 2015-03-01                                         5219.96   
3 2015-04-01                                         5011.69   
4 2015-05-01                                         5070.42   

   Leroy Domaine d'Auvenay Chevalier-Montrachet Grand Cru, Cote de Beaune, France  \
0                                            1692.71                                
1                                            1866.48                                
2                                            2104.56                                
3                                            2076.25                                
4                                            2253.69                                

   Leroy Domaine d'Auvenay Criots-Batard-Montrachet Grand Cru, Cote de Beaune, France  \
0              

/var/folders/bc/r66pqkps5_g70qndx3pw975r0000gn/T/ipykernel_19013/66957293.py:6: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



# 3. Explore Data Analysis (EDA)

## Plot original price

In [45]:
import ipywidgets as widgets
from IPython.display import display
import numpy as np
from matplotlib.ticker import FuncFormatter
import plotly.graph_objects as go

# Get the list of wine columns (all columns except the first one)
wine_columns = df.columns[1:]

# Create a dropdown widget for wine selection
dropdown = widgets.Dropdown(
    options=wine_columns,
    description='Wine:',
    style={'description_width': 'initial'},
    layout={'width': '800px'}
)

# Function to update the plot based on selected wine
def update_plot(wine):
    fig = go.Figure()
    
    # Add the selected wine price trend
    fig.add_trace(go.Scatter(
        x=df['Datetime'], 
        y=df[wine], 
        mode='lines', 
        name=wine
    ))
    
    # Update layout with numeric format
    fig.update_layout(
        title=f'Price Trend for {wine}',
        xaxis_title='Date',
        yaxis_title='Price ($)',
        template='plotly_white',
        height=600,
        width=1000,
        yaxis=dict(
            tickformat=",.", # Format with comma for thousands and dot for decimals
            separatethousands=True
        )
    )
    
    fig.show()

# Display the dropdown and create the interactive plot
interactive_plot = widgets.interactive(update_plot, wine=dropdown)
display(interactive_plot)

# Create a summary plot of all wines 
def plot_all_wines():
    fig = go.Figure()
    
    # Add traces for all wines
    for wine in wine_columns:
        fig.add_trace(go.Scatter(
            x=df['Datetime'],
            y=df[wine],
            mode='lines',
            name=wine
        ))
    
    # Update layout
    fig.update_layout(
        title='Price Trends for All Wines',
        xaxis_title='Date',
        yaxis_title='Price ($)',
        template='plotly_white',
        height=800,
        width=1200,
        yaxis=dict(
            tickformat=",.2f",
            separatethousands=True
        )
    )
    fig.show()

interactive(children=(Dropdown(description='Wine:', layout=Layout(width='800px'), options=('Leroy Musigny Gran…

## Plot log monthly price

In [46]:
# Create a dropdown widget for wine selection
dropdown = widgets.Dropdown(
    options=wine_columns,
    description='Wine:',
    style={'description_width': 'initial'},
    layout={'width': '800px'}
)

# Function to update the plot based on selected wine
def update_plot(wine):
    # Calculate log prices (handling potential zeros or negative values)
    prices = df[wine].replace(0, np.nan)  # Replace zeros with NaN
    log_prices = np.log(prices)
    
    fig = go.Figure()
    
    # Add the log price trend
    fig.add_trace(go.Scatter(
        x=df['Datetime'], 
        y=log_prices, 
        mode='lines', 
        name=f'Log Price of {wine}'
    ))
    
    # Update layout
    fig.update_layout(
        title=f'Monthly Log Prices for {wine}',
        xaxis_title='Date',
        yaxis_title='Log Price',
        template='plotly_white',
        height=600,
        width=1000
    )
    
    fig.show()

# Display the dropdown and create the interactive plot
interactive_plot = widgets.interactive(update_plot, wine=dropdown)
display(interactive_plot)

interactive(children=(Dropdown(description='Wine:', layout=Layout(width='800px'), options=('Leroy Musigny Gran…

## Plot return

In [47]:
# Create a dropdown widget for wine selection
dropdown = widgets.Dropdown(
    options=wine_columns,
    description='Wine:',
    style={'description_width': 'initial'},
    layout={'width': '800px'}
)

# Function to update the plot based on selected wine
def update_plot(wine):
    # Calculate daily returns (percentage change)
    returns = df[wine].pct_change().dropna()
    
    # Create a new dataframe with dates and returns (dropping the first row which has NaN return)
    returns_df = pd.DataFrame({
        'Date': df['Datetime'].iloc[1:],
        'Returns': returns
    })
    
    fig = go.Figure()
    
    # Add the returns trend
    fig.add_trace(go.Scatter(
        x=returns_df['Date'], 
        y=returns_df['Returns'], 
        mode='lines', 
        name=f'Returns of {wine}'
    ))
    
    # Add a horizontal line at y=0
    fig.add_shape(
        type="line",
        x0=returns_df['Date'].min(),
        y0=0,
        x1=returns_df['Date'].max(),
        y1=0,
        line=dict(color="black", width=1, dash="dash")
    )
    
    # Update layout
    fig.update_layout(
        title=f'Monthly Returns for {wine}',
        xaxis_title='Date',
        yaxis_title='Returns',
        template='plotly_white',
        height=600,
        width=1000
    )
    
    fig.show()

# Display the dropdown and create the interactive plot
interactive_plot = widgets.interactive(update_plot, wine=dropdown)
display(interactive_plot)

# Create a summary plot of all wines' returns
def plot_all_returns():
    fig = go.Figure()
    
    # Add traces for top 5 wines by average price
    top_wines = df[wine_columns].mean().sort_values(ascending=False).head(5).index
    
    for wine in top_wines:
        returns = df[wine].pct_change().dropna()
        dates = df['Datetime'].iloc[1:]  # Skip first date since return is NaN
        
        fig.add_trace(go.Scatter(
            x=dates, 
            y=returns, 
            mode='lines', 
            name=wine,
            opacity=0.7
        ))
    
    # Add a horizontal line at y=0
    fig.add_shape(
        type="line",
        x0=df['Datetime'].min(),
        y0=0,
        x1=df['Datetime'].max(),
        y1=0,
        line=dict(color="black", width=1, dash="dash")
    )

interactive(children=(Dropdown(description='Wine:', layout=Layout(width='800px'), options=('Leroy Musigny Gran…

## Time-series decomposition

In [48]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from statsmodels.tsa.seasonal import seasonal_decompose
import ipywidgets as widgets
from IPython.display import display, clear_output

# Assuming df is already defined and contains your wine price data
# Get all wine columns (excluding the Datetime column)
wine_columns = df.columns.drop('Datetime')
# Calculate returns for all wines
returns = df[wine_columns].pct_change().dropna()
# Store decompositions to avoid recalculating
decompositions = {}

# Pre-calculate decompositions for all wines
for wine in returns.columns:
    print(f"Pre-processing {wine}...")
    time_series = returns[wine]
    try:
        decomposition = seasonal_decompose(time_series, model='additive', period=12)
        decompositions[wine] = {
            'original': time_series,
            'trend': decomposition.trend,
            'seasonal': decomposition.seasonal,
            'residual': decomposition.resid
        }
    except Exception as e:
        print(f"Error processing {wine}: {e}")

# Create dropdown widget
wine_dropdown = widgets.Dropdown(
    options=list(decompositions.keys()),
    description='Select Wine:',
    style={'description_width': 'initial'},
    layout={'width': '500px'}
)

# Create output area for plots
output = widgets.Output()

# Function to create and display the plot
def create_decomposition_plot(wine):
    if wine not in decompositions:
        return None
    
    data = decompositions[wine]
    
    # Create subplots
    fig = make_subplots(
        rows=4, 
        cols=1,
        subplot_titles=(
            f'Original Time Series',
            f'Trend Component',
            f'Seasonal Component',
            f'Residual Component'
        ),
        vertical_spacing=0.1
    )
    # Get datetime values for x-axis
    datetime_values = df['Datetime']

    # Add original time series
    fig.add_trace(
        go.Scatter(
            x=datetime_values,
            y=data['original'].values,
            mode='lines',
            name='Original Time Series',
            line=dict(color='blue')
        ),
        row=1, col=1
    )

    # Add trend component
    fig.add_trace(
        go.Scatter(
            x=datetime_values,
            y=data['trend'].values,
            mode='lines',
            name='Trend Component',
            line=dict(color='orange')
        ),
        row=2, col=1
    )

    # Add seasonal component
    fig.add_trace(
        go.Scatter(
            x=datetime_values,
            y=data['seasonal'].values,
            mode='lines',
            name='Seasonal Component',
            line=dict(color='green')
        ),
        row=3, col=1
    )

    # Add residual component
    fig.add_trace(
        go.Scatter(
            x=datetime_values,
            y=data['residual'].values,
            mode='lines',
            name='Residual Component',
            line=dict(color='red')
        ),
        row=4, col=1
    )

    # Update layout
    fig.update_layout(
        height=800,
        width=1000,
        title_text=f"{wine} - Time Series Decomposition",
        showlegend=True,
        template='plotly_white'
    )

    # Update y-axis titles
    fig.update_yaxes(title_text="Returns Change", row=1, col=1)
    fig.update_yaxes(title_text="Trend", row=2, col=1)
    fig.update_yaxes(title_text="Seasonality", row=3, col=1)
    fig.update_yaxes(title_text="Residuals", row=4, col=1)

    # Update x-axis titles
    fig.update_xaxes(title_text="Date", row=4, col=1)
    
    return fig

# Function to handle dropdown change
def on_wine_change(change):
    with output:
        clear_output(wait=True)
        selected_wine = change.new
        fig = create_decomposition_plot(selected_wine)
        if fig:
            fig.show()
        else:
            print(f"No decomposition available for {selected_wine}")

# Connect the dropdown to the handler function
wine_dropdown.observe(on_wine_change, names='value')

# Display the dropdown and output area
display(wine_dropdown)
display(output)

# Initialize with the first wine
with output:
    initial_wine = wine_dropdown.value
    fig = create_decomposition_plot(initial_wine)
    if fig:
        fig.show()


Pre-processing Leroy Musigny Grand Cru, Cote de Nuits, France...
Pre-processing Leroy Domaine d'Auvenay Chevalier-Montrachet Grand Cru, Cote de Beaune, France...
Pre-processing Leroy Domaine d'Auvenay Criots-Batard-Montrachet Grand Cru, Cote de Beaune, France...
Pre-processing Leroy Chambertin Grand Cru, Cote de Nuits, France...
Pre-processing Leroy Domaine d'Auvenay Mazis-Chambertin Grand Cru, Cote de Nuits, France...
Pre-processing Leroy Domaine d'Auvenay Les Bonnes-Mares Grand Cru, Cote de Nuits, France...
Pre-processing Leroy Corton-Charlemagne Grand Cru, Cote de Beaune, France...
Pre-processing Domaine de la Romanee-Conti Montrachet Grand Cru, Cote de Beaune, France...
Pre-processing Leroy Richebourg Grand Cru, Cote de Nuits, France...
Pre-processing Leroy Romanee-Saint-Vivant Grand Cru, Cote de Nuits, France...
Pre-processing Leroy Domaine d'Auvenay Meursault Chaumes des Perrieres, Cote de Beaune, France...
Pre-processing Leroy Domaine d'Auvenay Meursault, Cote de Beaune, France.

Dropdown(description='Select Wine:', layout=Layout(width='500px'), options=('Leroy Musigny Grand Cru, Cote de …

Output()

# 4. Value at Risk (VaR) calculation

## Returns distribution

In [49]:
# Create a dropdown widget for wine selection
dropdown = widgets.Dropdown(
    options=wine_columns,
    description='Wine:',
    style={'description_width': 'initial'},
    layout={'width': '800px'}
)

# Function to create multi-panel histogram of returns at different historical windows
def plot_returns_windows(wine):
    # Calculate returns
    returns = df[wine].pct_change().dropna()
    
    # Determine the period length (assuming quarterly data, adjust as needed)
    # For wine data, we might need to adjust these periods based on actual data frequency
    period_length = len(returns) // 4  # Divide data into quarters
    
    # Create subplots
    fig = make_subplots(rows=3, cols=1, 
                        shared_yaxes=True,
                        subplot_titles=(f'Recent period (last {period_length} observations)',
                                       f'Mid period (observations {period_length+1} to {2*period_length})',
                                       f'Early period (observations {2*period_length+1} to {3*period_length})'))
    
    # Recent period
    history1 = returns[-period_length:]
    percentile_5_1 = np.percentile(history1, 5)
    percentile_1_1 = np.percentile(history1, 1)
    
    fig.add_trace(
        go.Histogram(x=history1, nbinsx=50, name='Recent period'),
        row=1, col=1
    )
    
    fig.add_vline(x=percentile_5_1, line=dict(color="orange", width=2), 
                 annotation_text="5%", annotation_position="top right", row=1, col=1)
    fig.add_vline(x=percentile_1_1, line=dict(color="red", width=2), 
                 annotation_text="1%", annotation_position="top right", row=1, col=1)
    
    # Mid period
    if len(returns) > period_length:
        start_idx = max(0, len(returns) - 2*period_length)
        end_idx = max(0, len(returns) - period_length)
        history2 = returns[start_idx:end_idx]
        
        if len(history2) > 0:
            percentile_5_2 = np.percentile(history2, 5)
            percentile_1_2 = np.percentile(history2, 1)
            
            fig.add_trace(
                go.Histogram(x=history2, nbinsx=50, name='Mid period'),
                row=2, col=1
            )
            
            fig.add_vline(x=percentile_5_2, line=dict(color="orange", width=2), 
                         annotation_text="5%", annotation_position="top right", row=2, col=1)
            fig.add_vline(x=percentile_1_2, line=dict(color="red", width=2), 
                         annotation_text="1%", annotation_position="top right", row=2, col=1)
    
    # Early period
    if len(returns) > 2*period_length:
        start_idx = max(0, len(returns) - 3*period_length)
        end_idx = max(0, len(returns) - 2*period_length)
        history3 = returns[start_idx:end_idx]
        
        if len(history3) > 0:
            percentile_5_3 = np.percentile(history3, 5)
            percentile_1_3 = np.percentile(history3, 1)
            
            fig.add_trace(
                go.Histogram(x=history3, nbinsx=50, name='Early period'),
                row=3, col=1
            )
            
            fig.add_vline(x=percentile_5_3, line=dict(color="orange", width=2), 
                         annotation_text="5%", annotation_position="top right", row=3, col=1)
            fig.add_vline(x=percentile_1_3, line=dict(color="red", width=2), 
                         annotation_text="1%", annotation_position="top right", row=3, col=1)
    
    # Update layout
    fig.update_layout(
        title_text=f"{wine} Returns Distribution at Different Historical Windows",
        height=800,
        width=1000,
        showlegend=False
    )
    
    # Update y-axis titles
    fig.update_yaxes(title_text="Frequency", row=2, col=1)
    
    # Update x-axis titles
    fig.update_xaxes(title_text="Returns", row=3, col=1)
    
    fig.show()

# Display the dropdown
display(dropdown)

# Create a button to show the multi-panel histogram
hist_button = widgets.Button(description="Show Returns Distribution by Time Period")
display(hist_button)

# Define the button click event
def on_hist_clicked(b):
    selected_wine = dropdown.value
    plot_returns_windows(selected_wine)
    
hist_button.on_click(on_hist_clicked)

# Additional function to show basic returns plot
def plot_basic_returns(wine):
    # Calculate daily returns
    returns = df[wine].pct_change().dropna()
    
    # Create a new dataframe with dates and returns
    returns_df = pd.DataFrame({
        'Date': df['Datetime'].iloc[1:],
        'Returns': returns
    })
    
    fig = go.Figure()
    
    # Add the returns trend
    fig.add_trace(go.Scatter(
        x=returns_df['Date'], 
        y=returns_df['Returns'], 
        mode='lines', 
        name=f'Returns of {wine}'
    ))


Dropdown(description='Wine:', layout=Layout(width='800px'), options=('Leroy Musigny Grand Cru, Cote de Nuits, …

Button(description='Show Returns Distribution by Time Period', style=ButtonStyle())

## HS VaR calculation

In [52]:
# Create a dropdown widget for wine selection
dropdown = widgets.Dropdown(
    options=wine_columns,
    description='Wine:',
    style={'description_width': 'initial'},
    layout={'width': '800px'}
)

# Function to calculate Historical VaR
def hs(returns, alpha=95, time_frame='month'):
    '''Calculates Historical VaR for monthly data'''
    if time_frame == 'month':
        time = 1
    elif time_frame == 'year':
        time = 12
    else:
        raise ValueError("'time_frame' input value must be one of {'month','year'}")
        
    q = 100 - alpha
    # Use numpy's percentile function with linear interpolation
    var = -np.sqrt(time) * np.percentile(returns, q)
    return np.round(var, 4)

# Function to calculate VaR and P&L for selected wine
def calculate_var_and_pnl(wine, investment=1000, prob=95):
    # Calculate returns
    returns = df[wine].pct_change().dropna()
    print(f"Returns for {wine}:")
    print(returns.head())
    
    # Calculate P&L in dollar terms
    pnl = investment * returns
    
    # Portfolio VaR calculation for different time horizons
    try:
        varM = investment * hs(returns, alpha=prob, time_frame='month')
        varY = investment * hs(returns, alpha=prob, time_frame='year')
        
        # Put results in dataframe
        var_df = pd.DataFrame({
            'HS VaR ($)': ['Monthly', 'Annual'],
            wine: [varM, varY]
        })
        
        return var_df, returns, pnl
    except Exception as e:
        print(f"Error calculating VaR: {e}")
        print(f"Returns shape: {returns.shape}, Returns type: {type(returns)}")
        return None, returns, None

# Function to display VaR, returns distribution, and P&L analysis
def show_var_analysis(wine, investment=1000, prob=95):
    # Calculate VaR and get returns
    var_df, returns, pnl = calculate_var_and_pnl(wine, investment, prob)
    
    if var_df is None:
        print(f"Unable to calculate VaR for {wine}")
        return
    
    # Create figure with subplots
    fig = make_subplots(
        rows=3, cols=1,
        subplot_titles=(
            f"Returns Distribution for {wine}", 
            f"Profit & Loss Distribution for {wine} (Investment: ${investment})",
            f"Value at Risk (VaR) for {wine}"
        ),
        specs=[[{"type": "scatter"}], [{"type": "scatter"}], [{"type": "table"}]],
        vertical_spacing=0.1,
        row_heights=[0.4, 0.4, 0.2]
    )
    
    # Add histogram of returns
    fig.add_trace(
        go.Histogram(
            x=returns,
            nbinsx=50,
            name='Returns Distribution',
            marker_color='blue',
            opacity=0.7
        ),
        row=1, col=1
    )
    
    # Add histogram of P&L
    fig.add_trace(
        go.Histogram(
            x=pnl,
            nbinsx=50,
            name='P&L Distribution',
            marker_color='green',
            opacity=0.7
        ),
        row=2, col=1
    )
    
    # Add vertical lines for VaR on returns plot
    q = 100 - prob
    var_line = np.percentile(returns, q)
    fig.add_vline(
        x=var_line,
        line=dict(color="red", width=2, dash="dash"),
        annotation_text=f"{prob}% VaR",
        annotation_position="top right",
        row=1, col=1
    )
    
    # Add vertical lines for VaR on P&L plot
    pnl_var_line = np.percentile(pnl, q)
    fig.add_vline(
        x=pnl_var_line,
        line=dict(color="red", width=2, dash="dash"),
        annotation_text=f"{prob}% VaR (${abs(pnl_var_line):.2f})",
        annotation_position="top right",
        row=2, col=1
    )
    
    # Add 1% and 5% percentile lines for returns
    p5 = np.percentile(returns, 5)
    p1 = np.percentile(returns, 1)
    
    fig.add_vline(
        x=p5,
        line=dict(color="orange", width=2),
        annotation_text="5%",
        annotation_position="top right",
        row=1, col=1
    )
    
    fig.add_vline(
        x=p1,
        line=dict(color="red", width=2),
        annotation_text="1%",
        annotation_position="top right",
        row=1, col=1
    )
    
    # Add 1% and 5% percentile lines for P&L
    p5_pnl = np.percentile(pnl, 5)
    p1_pnl = np.percentile(pnl, 1)
    
    fig.add_vline(
        x=p5_pnl,
        line=dict(color="orange", width=2),
        annotation_text=f"5% (${p5_pnl:.2f})",
        annotation_position="top right",
        row=2, col=1
    )
    
    fig.add_vline(
        x=p1_pnl,
        line=dict(color="red", width=2),
        annotation_text=f"1% (${p1_pnl:.2f})",
        annotation_position="top right",
        row=2, col=1
    )
    
    # Add zero line on P&L plot
    fig.add_vline(
        x=0,
        line=dict(color="black", width=1),
        row=2, col=1
    )
    
    # Add VaR table
    fig.add_trace(
        go.Table(
            header=dict(
                values=list(var_df.columns),
                fill_color='paleturquoise',
                align='center',
                font=dict(size=14)
            ),
            cells=dict(
                values=[var_df[col] for col in var_df.columns],
                fill_color='lavender',
                align='center',
                font=dict(size=12)
            )
        ),
        row=3, col=1
    )
    
    # Calculate P&L statistics
    pnl_mean = pnl.mean()
    pnl_std = pnl.std()
    pnl_min = pnl.min()
    pnl_max = pnl.max()
    pnl_pos_prob = (pnl > 0).mean() * 100
    
    # Add P&L statistics annotation
    pnl_stats = (
        f"P&L Statistics:<br>"
        f"Mean: ${pnl_mean:.2f}<br>"
        f"Std Dev: ${pnl_std:.2f}<br>"
        f"Min: ${pnl_min:.2f}<br>"
        f"Max: ${pnl_max:.2f}<br>"
        f"Prob. of Profit: {pnl_pos_prob:.1f}%"
    )
    
    fig.add_annotation(
        x=0.95,
        y=0.95,
        xref="paper",
        yref="paper",
        text=pnl_stats,
        showarrow=False,
        font=dict(size=12),
        bgcolor="white",
        bordercolor="black",
        borderwidth=1,
        borderpad=4,
        align="left"
    )
    
    # Update layout
    fig.update_layout(
        title_text=f"{wine}\n(Investment: ${investment}, Confidence: {prob}%)",
        height=1000, 
        width=1000, 
        showlegend=False
    )
    
    # Update y-axis titles
    fig.update_yaxes(title_text="Frequency", row=1, col=1)
    fig.update_yaxes(title_text="Frequency", row=2, col=1)
    
    # Update x-axis titles
    fig.update_xaxes(title_text="Returns", row=1, col=1)
    fig.update_xaxes(title_text="Profit/Loss ($)", row=2, col=1)
    
    fig.show()
    
    # Print backtesting results
    print("\nVaR Backtesting Results:")
    var_violations = (pnl < pnl_var_line).sum()
    total_observations = len(pnl)
    expected_violations = total_observations * (1 - prob/100)
    
    print(f"Expected VaR violations at {prob}% confidence: {expected_violations:.1f} ({(1-prob/100)*100:.1f}% of observations)")
    print(f"Actual VaR violations: {var_violations} ({var_violations/total_observations*100:.1f}% of observations)")
    
    if var_violations > expected_violations:
        print("Model may be underestimating risk (too many violations)")
    elif var_violations < expected_violations:
        print("Model may be overestimating risk (too few violations)")
    else:
        print("Model appears to be well-calibrated")

# Create input widgets
investment_input = widgets.IntText(
    value=1000,
    description='Investment ($):',
    style={'description_width': 'initial'},
    layout={'width': '300px'}
)

confidence_input = widgets.IntSlider(
    value=95,
    min=90,
    max=99,
    step=1,
    description='Confidence (%):',
    style={'description_width': 'initial'},
    layout={'width': '300px'}
)

# Display widgets
display(dropdown)
display(investment_input)
display(confidence_input)

# Create button
var_button = widgets.Button(description="Calculate VaR for Selected Wine")

# Display button
display(var_button)

# Define button click event
def on_var_clicked(b):
    selected_wine = dropdown.value
    investment = investment_input.value
    prob = confidence_input.value
    show_var_analysis(selected_wine, investment, prob)
    
var_button.on_click(on_var_clicked)


Dropdown(description='Wine:', layout=Layout(width='800px'), options=('Leroy Musigny Grand Cru, Cote de Nuits, …

IntText(value=1000, description='Investment ($):', layout=Layout(width='300px'), style=DescriptionStyle(descri…

IntSlider(value=95, description='Confidence (%):', layout=Layout(width='300px'), max=99, min=90, style=SliderS…

Button(description='Calculate VaR for Selected Wine', style=ButtonStyle())

# 5. ARIMA & SARIMA

## Training the model

In [58]:

# Assuming df is already defined and contains your wine price data
# First, make sure we have a proper datetime index for time series analysis

# Store predictions and evaluations
predictions = {}
results = {}

# Function to generate future dates
def generate_future_dates(last_date, steps=12):
    # For monthly data
    future_dates = pd.date_range(start=last_date, periods=steps+1, freq='MS')[1:]
    return future_dates

# Pre-calculate predictions for all wines
for wine in df.columns:
    if wine == 'Datetime':  # Skip the Datetime column if it exists
        continue
        
    print(f"Processing {wine}...")
    
    try:
        # Create a proper time series with datetime index
        if 'Datetime' in df.columns:
            # If we have a Datetime column, use it as index
            time_series = df[wine].copy()
            time_series.index = pd.to_datetime(df['Datetime'])
        else:
            # If we don't have a Datetime column, assume the index is already datetime
            time_series = df[wine].copy()
            
        # Calculate returns if needed
        returns = time_series.pct_change().dropna()
        
        # Make sure index is datetime and sorted
        returns.index = pd.to_datetime(returns.index)
        returns = returns.sort_index()
        
        # Fit ARIMA model
        arima_model = ARIMA(returns, order=(1, 1, 1))
        arima_fit = arima_model.fit()

        # Fit SARIMA model
        sarima_model = SARIMAX(returns, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
        sarima_fit = sarima_model.fit(disp=False)

        # Generate forecasts
        arima_forecast = arima_fit.get_forecast(steps=12)
        arima_forecast_series = arima_forecast.predicted_mean
        arima_forecast_ci = arima_forecast.conf_int()

        sarima_forecast = sarima_fit.get_forecast(steps=12)
        sarima_forecast_series = sarima_forecast.predicted_mean
        sarima_forecast_ci = sarima_forecast.conf_int()

        # Store predictions
        predictions[wine] = {
            'original': returns,
            'arima_forecast': arima_forecast_series,
            'arima_ci_lower': arima_forecast_ci.iloc[:, 0],
            'arima_ci_upper': arima_forecast_ci.iloc[:, 1],
            'sarima_forecast': sarima_forecast_series,
            'sarima_ci_lower': sarima_forecast_ci.iloc[:, 0],
            'sarima_ci_upper': sarima_forecast_ci.iloc[:, 1],
            'future_dates': arima_forecast_series.index  # Use the index from forecast
        }
        
        # For evaluation, we'll use the last 12 months as a test set
        if len(returns) > 24:  # Need enough data for training and testing
            train = returns[:-12]
            test = returns[-12:]
            
            # Fit models on training data
            arima_model_eval = ARIMA(train, order=(1, 1, 1))
            arima_fit_eval = arima_model_eval.fit()
            
            sarima_model_eval = SARIMAX(train, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
            sarima_fit_eval = sarima_model_eval.fit(disp=False)
            
            # Generate forecasts for test period
            arima_forecast_eval = arima_fit_eval.get_forecast(steps=12)
            arima_forecast_series_eval = arima_forecast_eval.predicted_mean
            
            sarima_forecast_eval = sarima_fit_eval.get_forecast(steps=12)
            sarima_forecast_series_eval = sarima_forecast_eval.predicted_mean
            
            # Align forecast with test data index
            arima_forecast_series_eval.index = test.index
            sarima_forecast_series_eval.index = test.index
            
            # Calculate metrics
            arima_mae = mean_absolute_error(test, arima_forecast_series_eval)
            arima_mse = mean_squared_error(test, arima_forecast_series_eval)
            arima_rmse = np.sqrt(arima_mse)
            # Handle potential division by zero in MAPE calculation
            arima_mape = np.mean(np.abs((test - arima_forecast_series_eval) / np.where(test == 0, 1e-10, test))) * 100
            
            sarima_mae = mean_absolute_error(test, sarima_forecast_series_eval)
            sarima_mse = mean_squared_error(test, sarima_forecast_series_eval)
            sarima_rmse = np.sqrt(sarima_mse)
            sarima_mape = np.mean(np.abs((test - sarima_forecast_series_eval) / np.where(test == 0, 1e-10, test))) * 100
            
            # Store evaluation results
            results[wine] = {
                'ARIMA': {'MAE': arima_mae, 'MSE': arima_mse, 'RMSE': arima_rmse, 'MAPE': arima_mape},
                'SARIMA': {'MAE': sarima_mae, 'MSE': sarima_mse, 'RMSE': sarima_rmse, 'MAPE': sarima_mape}
            }
        else:
            # print(f"Warning: {wine} has fewer than 24 data points, skipping evaluation")
            results[wine] = {
                'ARIMA': {'MAE': None, 'MSE': None, 'RMSE': None, 'MAPE': None},
                'SARIMA': {'MAE': None, 'MSE': None, 'RMSE': None, 'MAPE': None}
            }
            
    except Exception as e:
        print(f"Error processing {wine}: {e}")


/opt/homebrew/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

/opt/homebrew/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

/opt/homebrew/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

/opt/homebrew/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

/opt/homebrew/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

/opt/homebrew/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency informatio

## Plot and evaluate

In [ ]:

# Create dropdown widget for wine selection
wine_dropdown = widgets.Dropdown(
    options=[wine for wine in predictions.keys()],
    description='Select Wine:',
    style={'description_width': 'initial'},
    layout={'width': '500px'}
)

# Create dropdown for forecast horizon
horizon_dropdown = widgets.Dropdown(
    options=list(range(1, 13)),
    value=12,
    description='Forecast Months:',
    style={'description_width': 'initial'},
    layout={'width': '200px'}
)

# Create output area for plots
output = widgets.Output()

# Function to create and display the forecast plot
def create_forecast_plot(wine, horizon):
    if wine not in predictions:
        return None
    
    data = predictions[wine]
    
    # Create figure
    fig = go.Figure()
    
    # Add original time series
    fig.add_trace(
        go.Scatter(
            x=data['original'].index,
            y=data['original'].values,
            mode='lines',
            name='Historical Data',
            line=dict(color='blue')
        )
    )

    # Add ARIMA forecast
    fig.add_trace(
        go.Scatter(
            x=data['future_dates'][:horizon],
            y=data['arima_forecast'].values[:horizon],
            mode='lines',
            name='ARIMA Forecast',
            line=dict(color='orange')
        )
    )
    
    # Add ARIMA confidence intervals
    fig.add_trace(
        go.Scatter(
            x=list(data['future_dates'][:horizon]) + list(data['future_dates'][:horizon][::-1]),
            y=list(data['arima_ci_upper'].values[:horizon]) + list(data['arima_ci_lower'].values[:horizon][::-1]),
            fill='toself',
            fillcolor='rgba(255, 165, 0, 0.2)',
            line=dict(color='rgba(255, 165, 0, 0)'),
            name='ARIMA 95% CI'
        )
    )

    # Add SARIMA forecast
    fig.add_trace(
        go.Scatter(
            x=data['future_dates'][:horizon],
            y=data['sarima_forecast'].values[:horizon],
            mode='lines',
            name='SARIMA Forecast',
            line=dict(color='green')
        )
    )
    
    # Add SARIMA confidence intervals
    fig.add_trace(
        go.Scatter(
            x=list(data['future_dates'][:horizon]) + list(data['future_dates'][:horizon][::-1]),
            y=list(data['sarima_ci_upper'].values[:horizon]) + list(data['sarima_ci_lower'].values[:horizon][::-1]),
            fill='toself',
            fillcolor='rgba(0, 128, 0, 0.2)',
            line=dict(color='rgba(0, 128, 0, 0)'),
            name='SARIMA 95% CI'
        )
    )

    # Update layout
    fig.update_layout(
        height=600,
        width=1000,
        title_text=f"{wine} - {horizon} Month{'s' if horizon > 1 else ''} Forecast",
        xaxis_title="Date",
        yaxis_title="Price Change",
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
        template='plotly_white'
    )
    
    # Add model evaluation metrics
    if wine in results and results[wine]['ARIMA']['MAE'] is not None:
        metrics_text = (
            f"<b>Model Evaluation Metrics:</b><br>"
            f"ARIMA:  "
            f"RMSE={results[wine]['ARIMA']['RMSE']:.4f}<br>"
            f"SARIMA: "
            f"RMSE={results[wine]['SARIMA']['RMSE']:.4f}"
        )
        
        fig.add_annotation(
            xref="paper", yref="paper",
            x=0.5, y=-0.25,
            text=metrics_text,
            showarrow=False,
            font=dict(size=12),
            align="center"
        )
        
        # Adjust margins to make room for the metrics text
        fig.update_layout(margin=dict(b=100))
    
    return fig

# Function to handle dropdown changes
def on_change(change):
    with output:
        clear_output(wait=True)
        selected_wine = wine_dropdown.value
        selected_horizon = horizon_dropdown.value
        fig = create_forecast_plot(selected_wine, selected_horizon)
        if fig:
            fig.show()
        else:
            print(f"No forecast available for {selected_wine}")

# Connect the dropdowns to the handler function
wine_dropdown.observe(on_change, names='value')
horizon_dropdown.observe(on_change, names='value')

# Create a horizontal layout for the dropdowns
dropdown_layout = widgets.HBox([wine_dropdown, horizon_dropdown])

# Display the widgets
display(dropdown_layout)
display(output)

# Initialize with the first wine
with output:
    if len(predictions) > 0:
        initial_wine = wine_dropdown.value
        initial_horizon = horizon_dropdown.value
        fig = create_forecast_plot(initial_wine, initial_horizon)
        if fig:
            fig.show()
    else:
        print("No predictions available. Check the error messages above.")


Output()

In [31]:
# Create an empty DataFrame to store the results
results_df = pd.DataFrame(columns=['Wine','ARIMA RMSE','SARIMA RMSE'])
                                 
# Populate the DataFrame
for wine, metrics in results.items():
    row = {
        'Wine': wine,
        'ARIMA RMSE': metrics['ARIMA']['RMSE'],
        'SARIMA RMSE': metrics['SARIMA']['RMSE'],
    }
    results_df = pd.concat([results_df, pd.DataFrame([row])], ignore_index=True)

# Set Wine as index and display the first few rows
results_df = results_df.set_index('Wine')
results_df.head()

/var/folders/bc/r66pqkps5_g70qndx3pw975r0000gn/T/ipykernel_19013/1057199195.py:11: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



,ARIMA RMSE,SARIMA RMSE
Wine,,
"Leroy Musigny Grand Cru, Cote de Nuits, France",0.118706,0.118448
"Leroy Domaine d'Auvenay Chevalier-Montrachet Grand Cru, Cote de Beaune, France",0.029813,0.043800
"Leroy Domaine d'Auvenay Criots-Batard-Montrachet Grand Cru, Cote de Beaune, France",0.067584,0.085238
"Leroy Chambertin Grand Cru, Cote de Nuits, France",0.041694,0.042145
"Leroy Domaine d'Auvenay Mazis-Chambertin Grand Cru, Cote de Nuits, France",0.035529,0.049947


# 6. Long-Short-term memory

In [ ]:
def calculate_95ci(predictions):
    mean_pred = np.mean(predictions, axis=0)
    std_pred = np.std(predictions, axis=0)
    ci_lower = mean_pred - 1.96 * std_pred
    ci_upper = mean_pred + 1.96 * std_pred
    return ci_lower, ci_upper

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # Build, Fit, Predict and Evaluate the LSTM Model
        model = tf.keras.Sequential([
        LSTM(50, activation='relu', input_shape=(lag, 1)),
        tf.keras.layers.Dense(1)
    ])
        model.compile(optimizer='adam', loss='mse')
        model.summary()

# Define the model
model = LSTMModel(input_size=10, hidden_size=20, output_size=1)

# Set the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [33]:
# Assuming df and wine_columns are already defined
# Calculate percentage changes for all wine columns
returns = df[wine_columns].pct_change().dropna()

# Define a function to calculate 95% confidence interval
def calculate_95ci(predictions, alpha=0.05):
    n = len(predictions)
    std_err = np.std(predictions) / np.sqrt(n)
    h = std_err * 1.96  # 95% confidence interval
    return predictions - h, predictions + h

def calculate_99ci(predictions, alpha=0.01):
    n = len(predictions)
    std_err = np.std(predictions) / np.sqrt(n)
    h = std_err * 2.576  # 95% confidence interval
    return predictions - h, predictions + h

# Define root mean squared error function
def root_mean_squared_error(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# Create a dictionary to store models and results
wine_models = {}

# Process each wine and store the results
for wine in returns.columns:
    print(f"Processing {wine}...")
    time_series = returns[wine]
    
    # Prepare Data for LSTM
    scaler = StandardScaler()
    time_series = scaler.fit_transform(time_series.values.reshape(-1, 1))
    
    def create_lagged_features(data, lag):
        X, y = [], []
        for i in range(len(data) - lag):
            X.append(data[i:i+lag])
            y.append(data[i+lag])
        return np.array(X), np.array(y)

    lag = 12  # Number of past observations to use for prediction
    X, y = create_lagged_features(time_series, lag)
    X = X.reshape(X.shape[0], X.shape[1], 1)

    # Split into training and testing sets
    train_size = int(0.80 * len(X))
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    # Build, Fit, Predict and Evaluate the LSTM Model
    model = tf.keras.Sequential([
        LSTM(50, activation='relu', input_shape=(lag, 1), return_sequences=True),
        LSTM(50, activation='relu'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    
    # Define early stopping callback
    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    # Fit the model with early stopping
    model.fit(X_train, y_train, epochs=50, batch_size=16, verbose=0, validation_split=0.1, callbacks=[early_stop])

    y_pred_lstm = model.predict(X_test, verbose=0)
    y_pred_lstm_inverse = scaler.inverse_transform(y_pred_lstm)  # Inverse scaling for comparison
    y_test_inverse = scaler.inverse_transform(y_test.reshape(-1, 1))

    # Reconstruct training predictions for plotting
    train_predictions = model.predict(X_train, verbose=0)
    train_predictions_inverse = scaler.inverse_transform(train_predictions)

    # Calculate metrics
    mse = mean_squared_error(y_test_inverse, y_pred_lstm_inverse)
    rmse = root_mean_squared_error(y_test_inverse, y_pred_lstm_inverse)

    ci_lower_95, ci_upper_95 = calculate_95ci(y_pred_lstm_inverse)
    ci_lower_99, ci_upper_99 = calculate_99ci(y_pred_lstm_inverse)
    
    # Function to predict future values
    def predict_future(model, last_sequence, steps, scaler):
        future_predictions = []
        current_sequence = last_sequence.copy()
        
        for _ in range(steps):
            # Reshape for model input
            current_input = current_sequence.reshape(1, lag, 1)
            # Get prediction
            next_pred = model.predict(current_input, verbose=0)[0][0]
            # Add to predictions
            future_predictions.append(next_pred)
            # Update sequence
            current_sequence = np.append(current_sequence[1:], next_pred)
        
        # Convert predictions back to original scale
        future_predictions = np.array(future_predictions).reshape(-1, 1)
        future_predictions_inverse = scaler.inverse_transform(future_predictions)
        
        return future_predictions_inverse
    
    # Get the last sequence from the data
    last_sequence = time_series[-lag:]
    
    # Predict 12 months (1 year) ahead
    future_12m = predict_future(model, last_sequence, 12, scaler)
    
    # Calculate confidence intervals for future predictions
    future_lower, future_upper = calculate_95ci(future_12m)

    # Store all the necessary data for plotting later
    wine_models[wine] = {
        'time_series': time_series,
        'scaler': scaler,
        'lag': lag,
        'train_size': train_size,
        'train_predictions_inverse': train_predictions_inverse,
        'y_test_inverse': y_test_inverse,
        'y_pred_lstm_inverse': y_pred_lstm_inverse,
        'ci_lower_95': ci_lower_95,
        'ci_upper_95': ci_upper_95,
        'ci_lower_99': ci_lower_99,
        'ci_upper_99': ci_upper_99,
        'rmse': rmse,
        'future_predictions': future_12m,
        'future_lower': future_lower,
        'future_upper': future_upper
    }

# Create a dropdown widget for wine selection
wine_dropdown = widgets.Dropdown(
    options=wine_columns,
    description='Select Wine:',
    style={'description_width': 'initial'},
    layout={'width': '500px'}
)

# Create a radio button for forecast horizon
forecast_horizon = widgets.RadioButtons(
    options=['3 Months', '6 Months', '12 Months'],
    description='Display Horizon:',
    style={'description_width': 'initial'},
)

# Function to generate future month strings
def generate_future_month_strings(last_date, num_months):
    # Extract year and month from the last date
    if isinstance(last_date, pd.Timestamp):
        year = last_date.year
        month = last_date.month
    else:
        # Try to parse the date if it's not a Timestamp
        dt = pd.to_datetime(last_date)
        year = dt.year
        month = dt.month
    
    future_months = []
    for i in range(1, num_months + 1):
        # Calculate new month and year
        new_month = ((month - 1 + i) % 12) + 1
        new_year = year + ((month - 1 + i) // 12)
        
        # Format as string
        month_name = calendar.month_abbr[new_month]
        future_months.append(f"{month_name} {new_year}")
    
    return future_months

# Function to plot the selected wine's forecast
def plot_wine_forecast(wine, horizon):
    if wine not in wine_models:
        print(f"No model found for {wine}")
        return None, None
    
    model_data = wine_models[wine]
    
    # Get the data
    time_series = model_data['time_series']
    scaler = model_data['scaler']
    lag = model_data['lag']
    train_size = model_data['train_size']
    y_test_inverse = model_data['y_test_inverse']
    y_pred_lstm_inverse = model_data['y_pred_lstm_inverse']
    ci_lower_95 = model_data['ci_lower_95']
    ci_upper_95 = model_data['ci_upper_95']
    ci_lower_99 = model_data['ci_lower_99']
    ci_upper_99 = model_data['ci_upper_99']
    rmse = model_data['rmse']
    future_predictions = model_data['future_predictions']
    future_lower = model_data['future_lower']
    future_upper = model_data['future_upper']
    
    # Calculate indices for train and test data
    train_start_idx = lag
    train_end_idx = train_size + lag
    test_end_idx = train_end_idx + len(y_test_inverse)
    
    # Get the dates for historical data
    historical_dates = df['Datetime'].iloc[-len(time_series):].values
    train_dates = df['Datetime'].iloc[train_start_idx:train_end_idx].values
    test_dates = df['Datetime'].iloc[train_end_idx:test_end_idx].values
    
    # Get the last date in the dataset
    last_date = df['Datetime'].iloc[-1]
    
    # Determine how many months to display based on selected horizon
    if horizon == '3 Months':
        display_months = 3
    elif horizon == '6 Months':
        display_months = 6
    else:  # 12 Months
        display_months = 12
    
    # Generate future month strings (not datetime objects)
    future_month_strings = generate_future_month_strings(last_date, 12)
    
    # Create Plotly figure
    fig = go.Figure()
    
    # Add actual data
    fig.add_trace(go.Scatter(
        x=historical_dates, 
        y=scaler.inverse_transform(time_series.reshape(-1, 1)).flatten(),
        mode='lines',
        name='Historical Data',
        line=dict(color='blue')
    ))
    
    # Add true test values
    fig.add_trace(go.Scatter(
        x=test_dates, 
        y=y_test_inverse.flatten(),
        mode='lines',
        name='True Test Values',
        line=dict(color='green')
    ))
    
    # Add test predictions
    fig.add_trace(go.Scatter(
        x=test_dates, 
        y=y_pred_lstm_inverse.flatten(),
        mode='lines',
        name='Test Predictions',
        line=dict(color='red')
    ))
    
    # Add test 95 confidence interval - upper bound
    fig.add_trace(go.Scatter(
        x=test_dates,
        y=ci_upper_95.flatten(),
        mode='lines',
        line=dict(width=0),
        showlegend=False
    ))
    
    # Add test 95 confidence interval - lower bound
    fig.add_trace(go.Scatter(
        x=test_dates,
        y=ci_lower_95.flatten(),
        mode='lines',
        line=dict(width=0),
        fill='tonexty',
        fillcolor='rgba(255,165,0,0.2)',
        name='95% CI (Test)'
    ))
    
     # Add test 99 confidence interval - upper bound
    fig.add_trace(go.Scatter(
        x=test_dates,
        y=ci_upper_99.flatten(),
        mode='lines',
        line=dict(width=0),
        showlegend=False
    ))
    
    # Add test 99 confidence interval - lower bound
    fig.add_trace(go.Scatter(
        x=test_dates,
        y=ci_lower_99.flatten(),
        mode='lines',
        line=dict(width=0),
        fill='tonexty',
        fillcolor='rgba(256,165,0,0.2)',
        name='99% CI (Test)'
    ))
    # For the future predictions, we'll use a separate subplot since we're using string labels
    # Create a new figure for future predictions
    future_fig = go.Figure()
    
    # Add future predictions - only show the selected number of months
    display_month_strings = future_month_strings[:display_months]
    display_predictions = future_predictions[:display_months]
    display_lower_95 = future_lower_95[:display_months]
    display_upper_95 = future_upper_95[:display_months] 
    display_lower_99 = future_lower_99[:display_months]
    display_upper_99 = future_upper_99[:display_months]     
    
    # Add future predictions line
    future_fig.add_trace(go.Scatter(
        x=display_month_strings, 
        y=display_predictions.flatten(),
        mode='lines+markers',
        name=f'Future Predictions ({horizon})',
        line=dict(color='purple'),
        marker=dict(size=8)
    ))
    
    # Add future confidence interval - upper bound
    future_fig.add_trace(go.Scatter(
        x=display_month_strings,
        y=display_upper_95.flatten(),
        mode='lines',
        line=dict(width=0),
        showlegend=False
    ))
    
    # Add future 95 confidence interval - lower bound
    future_fig.add_trace(go.Scatter(
        x=display_month_strings,
        y=display_lower_95.flatten(),
        mode='lines',
        line=dict(width=0),
        fill='tonexty',
        fillcolor='rgba(128,0,128,0.2)',
        name=f'95% CI ({horizon})'
    ))

    # Add future 99 confidence interval - upper bound
    future_fig.add_trace(go.Scatter(
        x=display_month_strings,
        y=display_upper_99.flatten(),
        mode='lines',
        line=dict(width=0),
        showlegend=False
    ))
    
    # Add future 99 confidence interval - lower bound
    future_fig.add_trace(go.Scatter(
        x=display_month_strings,
        y=display_lower_99.flatten(),
        mode='lines',
        line=dict(width=0),
        fill='tonexty',
        fillcolor='rgba(127,0,128,0.2)',
        name=f'95% CI ({horizon})'
    ))
    
    # Update future figure layout
    future_fig.update_layout(
        title=f'{wine} - Future Predictions ({horizon})',
        xaxis_title='Month',
        yaxis_title='Returns Change',
        legend_title='Legend',
        height=400,
        width=800,
        template='plotly_white',
        hovermode='x unified'
    )
    
    # Create a table to show all 12 months of predictions
    table_fig = go.Figure(data=[go.Table(
        header=dict(values=["Month", "Predicted Value"],
                    fill_color='paleturquoise',
                    align='left'),
        cells=dict(values=[future_month_strings, 
                          [f"{val[0]:.4f}" for val in future_predictions]],
                  fill_color='lavender',
                  align='left')
    )])
    
    table_fig.update_layout(
        title=f'{wine} - Monthly Predictions (All 12 Months)',
        height=400,
        width=500
    )
    
    # Update historical figure layout
    fig.update_layout(
        title=f'{wine} - RMSE: {rmse:.3}',
        xaxis_title='Date',
        yaxis_title='Returns Change',
        legend_title='Legend',
        height=400,
        width=800,
        template='plotly_white',
        hovermode='x unified'
    )
    
    return fig, future_fig, table_fig

# Function to handle widget changes
def on_widget_change(change):
    with output:
        clear_output(wait=True)
        selected_wine = wine_dropdown.value
        selected_horizon = forecast_horizon.value
        try:
            fig, future_fig, table_fig = plot_wine_forecast(selected_wine, selected_horizon)
            if fig and future_fig and table_fig:
                fig.show()
                future_fig.show()
                table_fig.show()
        except Exception as e:
            print(f"Error generating plot: {e}")
            import traceback
            traceback.print_exc()

# Connect the widgets to the handler function
wine_dropdown.observe(on_widget_change, names='value')
forecast_horizon.observe(on_widget_change, names='value')

# Create output area
output = widgets.Output()

# Display the widgets and output area
display(widgets.HBox([wine_dropdown, forecast_horizon]))
display(output)

# Initialize with the first wine and 3-month forecast
with output:
    try:
        fig, future_fig, table_fig = plot_wine_forecast(wine_dropdown.value, forecast_horizon.value)
        if fig and future_fig and table_fig:
            fig.show()
            future_fig.show()
            table_fig.show()
    except Exception as e:
        print(f"Error generating initial plot: {e}")
        import traceback
        traceback.print_exc()

Processing Leroy Musigny Grand Cru, Cote de Nuits, France...


/opt/homebrew/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Processing Leroy Domaine d'Auvenay Chevalier-Montrachet Grand Cru, Cote de Beaune, France...


/opt/homebrew/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Processing Leroy Domaine d'Auvenay Criots-Batard-Montrachet Grand Cru, Cote de Beaune, France...


...

Processing Coche-Dury Corton-Charlemagne Grand Cru, Cote de Beaune, France...


/opt/homebrew/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Processing Leroy Domaine d'Auvenay Les Gouttes d'Or, Meursault Premier Cru, France...


/opt/homebrew/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Output()

In [34]:
# Get first wine's rmse value
first_wine = list(wine_models.keys())[0]
rmse_value = wine_models[first_wine]['rmse']
print(f"RMSE for {first_wine}: {rmse_value}")

# Print RMSE for all wines
for wine, data in wine_models.items():
	print(f"{wine}: {data['rmse']}")


RMSE for Leroy Musigny Grand Cru, Cote de Nuits, France: 0.11140792387798117
Leroy Musigny Grand Cru, Cote de Nuits, France: 0.11140792387798117
Leroy Domaine d'Auvenay Chevalier-Montrachet Grand Cru, Cote de Beaune, France: 0.05415177582021895
Leroy Domaine d'Auvenay Criots-Batard-Montrachet Grand Cru, Cote de Beaune, France: 0.06438916880592672
Leroy Chambertin Grand Cru, Cote de Nuits, France: 0.04817894156436039
Leroy Domaine d'Auvenay Mazis-Chambertin Grand Cru, Cote de Nuits, France: 0.04778480661717704
Leroy Domaine d'Auvenay Les Bonnes-Mares Grand Cru, Cote de Nuits, France: 0.03801056261949238
Leroy Corton-Charlemagne Grand Cru, Cote de Beaune, France: 0.04437751451789726
Domaine de la Romanee-Conti Montrachet Grand Cru, Cote de Beaune, France: 0.029032333196153254
Leroy Richebourg Grand Cru, Cote de Nuits, France: 0.027593439193382592
Leroy Romanee-Saint-Vivant Grand Cru, Cote de Nuits, France: 0.09323398187164998
Leroy Domaine d'Auvenay Meursault Chaumes des Perrieres, Cote 

In [35]:
# Create a new column for LSTM RMSE
lstm_rmse = []

# Extract RMSE values for each wine
for wine in results_df.index:
    if wine in wine_models:
        lstm_rmse.append(wine_models[wine]['rmse'])
    else:
        lstm_rmse.append(None)

# Add LSTM RMSE column to results_df
results_df['LSTM RMSE'] = lstm_rmse

# Sort by LSTM RMSE to see best performing models
results_df = results_df.sort_values('LSTM RMSE')

# Display the updated DataFrame
results_df.head()

,ARIMA RMSE,SARIMA RMSE,LSTM RMSE
Wine,,,
"Domaine de la Romanee-Conti La Tache Grand Cru Monopole, Cote de Nuits, France",0.015055,0.017337,0.021410
"Leroy Richebourg Grand Cru, Cote de Nuits, France",0.029579,0.031808,0.027593
"Domaine Jean-Louis Chave Ermitage 'Cuvee Cathelin', Rhone, France",0.027843,0.038166,0.028151
"Domaine de la Romanee-Conti Montrachet Grand Cru, Cote de Beaune, France",0.024578,0.026801,0.029032
"Leroy Clos de Vougeot Grand Cru, Cote de Nuits, France",0.029168,0.050274,0.030618


In [37]:
# Create the full file path by joining the directory path and filename
csv_path = os.path.join(path, 'wine_model_results_rmse.csv')

# Save the DataFrame to CSV
results_df.to_csv(csv_path, index=True)

print(f"Results saved to: {csv_path}")

Results saved to: /Users/esther/Desktop/202506final_thesis/VScode/wine_model_results_rmse.csv


# 7. DeepVaR

## DeepVaR training

In [7]:
path = '/Users/esther/Desktop/202506final_thesis/VScode'
MODEL_SAVE_DIR = os.path.join(path, 'saved_models')
os.makedirs(MODEL_SAVE_DIR, exist_ok=True)
print(MODEL_SAVE_DIR)

/Users/esther/Desktop/202506final_thesis/VScode/saved_models


In [10]:
# Assuming df and wine_columns are already defined
# Calculate percentage changes for all wine columns
returns = df[wine_columns].pct_change().dropna()

# Function to calculate HS VaR 95CI
def hs95(returns, alpha=95, time_frame='month'):
    '''Calculates Historical VaR for monthly data'''
    if time_frame == 'month':
        time = 1
    elif time_frame == 'year':
        time = 12
    else:
        raise ValueError("'time_frame' input value must be one of {'month','year'}")
        
    q = 100 - alpha
    # Use numpy's percentile function with linear interpolation
    var = -np.sqrt(time) * np.percentile(returns, q)
    return np.round(var, 4)

# Function to calculate VaR for selected wine
def calculate_var_95(wine, investment=1000, prob=95):
    # Calculate returns
    wine_returns = df[wine].pct_change().dropna()
    var_value = hs95(wine_returns, alpha=prob)
    var_amount = investment * var_value
    return var_value, var_amount

# Function to calculate 95% confidence interval
def calculate_95ci(predictions, alpha=0.05):
    n = len(predictions)
    std_err = np.std(predictions) / np.sqrt(n)
    h = std_err * 1.96  # 95% confidence interval
    return predictions - h, predictions + h

# Function to calculate HS VaR 99CI
# Function to calculate Historical VaR
def hs99(returns, alpha=99, time_frame='month'):
    '''Calculates Historical VaR for monthly data'''
    if time_frame == 'month':
        time = 1
    elif time_frame == 'year':
        time = 12
    else:
        raise ValueError("'time_frame' input value must be one of {'month','year'}")
        
    q = 100 - alpha
    # Use numpy's percentile function with linear interpolation
    var = -np.sqrt(time) * np.percentile(returns, q)
    return np.round(var, 4)

# Function to calculate VaR for selected wine
def calculate_var_99(wine, investment=1000, prob=99):
    # Calculate returns
    wine_returns = df[wine].pct_change().dropna()
    var_value = hs99(wine_returns, alpha=prob)
    var_amount = investment * var_value
    return var_value, var_amount

# Function to calculate 99% confidence interval
def calculate_99ci(predictions, alpha=0.01):
    n = len(predictions)
    std_err = np.std(predictions) / np.sqrt(n)
    h = std_err * 2.576  # 99% confidence interval (using z-score for 99% CI)
    return predictions - h, predictions + h

# Define root mean squared error function
def root_mean_squared_error(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# --- Helper Function for Filenames ---
def sanitize_filename(name):
    """Removes characters problematic for filenames and shortens."""
    # Remove or replace special characters
    name = re.sub(r'[,\'&]', '', name)
    name = re.sub(r'\s+', '_', name) # Replace spaces with underscores
    # Limit length to avoid issues on some file systems
    return name[:50]

# Create a dictionary to store models and results
wine_models = {}
path = '/Users/esther/Desktop/202506final_thesis/VScode'
MODEL_SAVE_DIR = os.path.join(path, 'saved_models')
os.makedirs(MODEL_SAVE_DIR, exist_ok=True)

# Process each wine and store the results
for wine in returns.columns:
    print(f"Processing {wine}...")
    time_series = returns[wine]
    
    # Calculate historical VaR
    historical_var_95, _ = calculate_var_95(wine)
    historical_var_99, _ = calculate_var_99(wine)
    
    # Prepare Data for LSTM
    scaler = StandardScaler()
    time_series = scaler.fit_transform(time_series.values.reshape(-1, 1))
    
    def create_lagged_features(data, lag):
        X, y = [], []
        for i in range(len(data) - lag):
            X.append(data[i:i+lag])
            y.append(data[i+lag])
        return np.array(X), np.array(y)

    lag = 12  # Number of past observations to use for prediction
    X, y = create_lagged_features(time_series, lag)
    X = X.reshape(X.shape[0], X.shape[1], 1)

    # Split into training and testing sets
    train_size = int(0.80 * len(X))
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    # Build, Fit, Predict and Evaluate the LSTM Model
    model = tf.keras.Sequential([
        LSTM(50, activation='relu', input_shape=(lag, 1), return_sequences=True),
        LSTM(50, activation='relu'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    
    # Define early stopping callback
    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    # Fit the model with early stopping
    model.fit(X_train, y_train, epochs=50, batch_size=16, verbose=0, validation_split=0.1, callbacks=[early_stop])

    y_pred_lstm = model.predict(X_test, verbose=0)
    y_pred_lstm_inverse = scaler.inverse_transform(y_pred_lstm)  # Inverse scaling for comparison
    y_test_inverse = scaler.inverse_transform(y_test.reshape(-1, 1))

    # Reconstruct training predictions for plotting
    train_predictions = model.predict(X_train, verbose=0)
    train_predictions_inverse = scaler.inverse_transform(train_predictions)

    # Calculate metrics
    mse = mean_squared_error(y_test_inverse, y_pred_lstm_inverse)
    rmse = root_mean_squared_error(y_test_inverse, y_pred_lstm_inverse)

    ci_lower_95, ci_upper_95 = calculate_95ci(y_pred_lstm_inverse)
    ci_lower_99, ci_upper_99 = calculate_99ci(y_pred_lstm_inverse)
    
    # Function to predict future values
    def predict_future(model, last_sequence, steps, scaler):
        future_predictions = []
        current_sequence = last_sequence.copy()
        
        for _ in range(steps):
            # Reshape for model input
            current_input = current_sequence.reshape(1, lag, 1)
            # Get prediction
            next_pred = model.predict(current_input, verbose=0)[0][0]
            # Add to predictions
            future_predictions.append(next_pred)
            # Update sequence
            current_sequence = np.append(current_sequence[1:], next_pred)
        
        # Convert predictions back to original scale
        future_predictions = np.array(future_predictions).reshape(-1, 1)
        future_predictions_inverse = scaler.inverse_transform(future_predictions)
        
        return future_predictions_inverse
    
    # Get the last sequence from the data
    last_sequence = time_series[-lag:]
    
    # Predict 12 months (1 year) ahead
    future_12m = predict_future(model, last_sequence, 12, scaler)
    
    # Calculate confidence intervals for future predictions
    future_lower_95, future_upper_95 = calculate_95ci(future_12m)
    future_lower_99, future_upper_99 = calculate_99ci(future_12m)
    
    # Calculate Deep VaR using LSTM predictions
    # Deep VaR uses the predicted returns from the LSTM model to calculate VaR
    # This provides a forward-looking VaR estimate based on the model's predictions
    deep_var_values_95 = []
    deep_var_amounts_95 = []
    deep_var_values_99 = []
    deep_var_amounts_99 = []
    investment_amount = 1000  # Default investment amount
    
    # For each month, we'll simulate potential returns based on the model's prediction and its uncertainty
    for i in range(len(future_12m)):
        # Get the predicted return and its confidence interval
        pred_return = future_12m[i][0]  # Extract scalar from array
        pred_lower_95 = future_lower_95[i][0]  # Extract scalar from array for 95% CI
        pred_lower_99 = future_lower_99[i][0]  # Extract scalar from array for 99% CI
        # Estimate the standard deviation based on the confidence interval
        std_dev_95 = (pred_return - pred_lower_95) / 1.96
        std_dev_99 = (pred_return - pred_lower_99) / 2.576
        
        
        # Generate a simulated distribution of returns centered on the prediction
        # with the estimated standard deviation
        num_simulations = 10000
        # Use absolute value of standard deviation to avoid ValueError: scale < 0
        std_dev_95_abs = abs(std_dev_95)
        std_dev_99_abs = abs(std_dev_99)
        simulated_returns_95 = np.random.normal(pred_return, std_dev_95_abs, num_simulations)
        simulated_returns_99 = np.random.normal(pred_return, std_dev_99_abs, num_simulations)
        
        # Calculate VaR 95CI from the simulated distribution
        var_value_95 = -np.percentile(simulated_returns_95, 5)  # 95% VaR
        var_amount_95 = investment_amount * var_value_95
        
        deep_var_values_95.append(var_value_95)
        deep_var_amounts_95.append(var_amount_95)

        # Calculate VaR 99CI from the simulated distribution
        var_value_99 = -np.percentile(simulated_returns_99, 1)  # 99% VaR
        var_amount_99 = investment_amount * var_value_99
        
        deep_var_values_99.append(var_value_99)
        deep_var_amounts_99.append(var_amount_99)
        
        deep_var_values_99.append(var_value_99)
        deep_var_amounts_99.append(var_amount_99)

    # try:
    #     sanitized_name = sanitize_filename(wine)
    #     model_path = os.path.join(MODEL_SAVE_DIR, f"model_{sanitized_name}.keras")
    #     scaler_path = os.path.join(MODEL_SAVE_DIR, f"scaler_{sanitized_name}.joblib")

    #     model.save(model_path)
    #     joblib.dump(scaler, scaler_path)

    #     print(f"Successfully saved model to: {model_path}")
    #     print(f"Successfully saved scaler to: {scaler_path}")

    # except Exception as e:
    #     print(f"!!! Error saving model or scaler for {wine}: {e}")

    # print("\n--- Model training and saving process complete. ---")

    # Store all the necessary data for plotting later
    wine_models[wine] = {
        'time_series': time_series,
        'scaler': scaler,
        'lag': lag,
        'train_size': train_size,
        'train_predictions_inverse': train_predictions_inverse,
        'y_test_inverse': y_test_inverse,
        'y_pred_lstm_inverse': y_pred_lstm_inverse,
        'ci_lower_95': ci_lower_95,
        'ci_upper_95': ci_upper_95,
        'rmse': rmse,
        'future_predictions': future_12m,
        'future_lower_95': future_lower_95,
        'future_upper_95': future_upper_95,
        'ci_lower_99': ci_lower_99,
        'ci_upper_99': ci_upper_99,
        'future_upper_99': future_upper_99, 
        'future_lower_99': future_lower_99,   
        'historical_var_95': historical_var_95,
        'deep_var_values_95': deep_var_values_95,
        'deep_var_amounts_95': deep_var_amounts_95,
        'historical_var_99': historical_var_99,
        'deep_var_values_99': deep_var_values_99,
        'deep_var_amounts_99': deep_var_amounts_99
    }


Processing Leroy Musigny Grand Cru, Cote de Nuits, France...


/opt/homebrew/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Processing Leroy Domaine d'Auvenay Chevalier-Montrachet Grand Cru, Cote de Beaune, France...


/opt/homebrew/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Processing Leroy Domaine d'Auvenay Criots-Batard-Montrachet Grand Cru, Cote de Beaune, France...


...

/opt/homebrew/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Processing Coche-Dury Corton-Charlemagne Grand Cru, Cote de Beaune, France...


/opt/homebrew/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Processing Leroy Domaine d'Auvenay Les Gouttes d'Or, Meursault Premier Cru, France...


/opt/homebrew/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



## Testing & benchmark with PnL

In [11]:

for wine in returns.columns:
    pass  # This will be handled by the widgets below

# Create widgets for interactive visualization
wine_dropdown = widgets.Dropdown(
    options=wine_columns,
    description='Select Wine:',
    style={'description_width': 'initial'},
    layout={'width': '500px'}
)

forecast_horizon = widgets.RadioButtons(
    options=['3 Months', '6 Months', '12 Months'],
    description='Display Horizon:',
    style={'description_width': 'initial'},
)

investment_slider = widgets.IntSlider(
    value=1000,
    min=100,
    max=10000,
    step=100,
    description='Investment ($):',
    style={'description_width': 'initial'},
    layout={'width': '500px'}
)

# Function to generate future month strings
def generate_future_month_strings(last_date, num_months):
    # Extract year and month from the last date
    if isinstance(last_date, pd.Timestamp):
        year = last_date.year
        month = last_date.month
    else:
        # Try to parse the date if it's not a Timestamp
        dt = pd.to_datetime(last_date)
        year = dt.year
        month = dt.month
    
    future_months = []
    for i in range(1, num_months + 1):
        # Calculate new month and year
        new_month = ((month - 1 + i) % 12) + 1
        new_year = year + ((month - 1 + i) // 12)
        
        # Format as string
        month_name = calendar.month_abbr[new_month]
        future_months.append(f"{month_name} {new_year}")
    
    return future_months

# Function to plot the selected wine's forecast and VaR
def plot_wine_forecast_and_var(wine, horizon, investment):
    if wine not in wine_models:
        print(f"No model found for {wine}")
        return None, None, None, None
    
    model_data = wine_models[wine]
    
    # Get the data
    time_series = model_data['time_series']
    scaler = model_data['scaler']
    lag = model_data['lag']
    train_size = model_data['train_size']
    y_test_inverse = model_data['y_test_inverse']
    y_pred_lstm_inverse = model_data['y_pred_lstm_inverse']
    ci_lower_95 = model_data['ci_lower_95']
    ci_upper_95 = model_data['ci_upper_95']
    ci_lower_99 = model_data['ci_lower_99']
    ci_upper_99 = model_data['ci_upper_99']
    rmse = model_data['rmse']
    future_predictions = model_data['future_predictions']
    future_lower_95 = model_data['future_lower_95']
    future_upper_95 = model_data['future_upper_95']
    future_lower_99 = model_data['future_lower_99']
    future_upper_99 = model_data['future_upper_99']
    historical_var_95 = model_data['historical_var_95']
    deep_var_values_95 = model_data['deep_var_values_95']
    deep_var_amounts_95 = model_data['deep_var_amounts_95']
    historical_var_99 = model_data['historical_var_99']
    deep_var_values_99 = model_data['deep_var_values_99']
    deep_var_amounts_99 = model_data['deep_var_amounts_99']
    
    # Calculate indices for train and test data
    train_start_idx = lag
    train_end_idx = train_size + lag
    test_end_idx = train_end_idx + len(y_test_inverse)
    
    # Get the dates for historical data
    historical_dates = df['Datetime'].iloc[-len(time_series):].values
    train_dates = df['Datetime'].iloc[train_start_idx:train_end_idx].values
    test_dates = df['Datetime'].iloc[train_end_idx:test_end_idx].values
    
    # Get the last date in the dataset
    last_date = df['Datetime'].iloc[-1]
    
    # Determine how many months to display based on selected horizon
    if horizon == '3 Months':
        display_months = 3
    elif horizon == '6 Months':
        display_months = 6
    else:  # 12 Months
        display_months = 12
    
    # Generate future month strings (not datetime objects)
    future_month_strings = generate_future_month_strings(last_date, 12)
    
    # Create Plotly figure for historical performance
    fig = go.Figure()
    
    # Add actual data
    fig.add_trace(go.Scatter(
        x=historical_dates, 
        y=scaler.inverse_transform(time_series.reshape(-1, 1)).flatten(),
        mode='lines',
        name='Historical Data',
        line=dict(color='blue')
    ))
    
    # Add true test values
    fig.add_trace(go.Scatter(
        x=test_dates, 
        y=y_test_inverse.flatten(),
        mode='lines',
        name='True Test Values',
        line=dict(color='green')
    ))
    
    # Add test predictions
    fig.add_trace(go.Scatter(
        x=test_dates, 
        y=y_pred_lstm_inverse.flatten(),
        mode='lines',
        name='Test Predictions',
        line=dict(color='red')
    ))
    
    # Add test 95 confidence interval - upper bound
    fig.add_trace(go.Scatter(
        x=test_dates,
        y=ci_upper_95.flatten(),
        mode='lines',
        line=dict(width=0),
        showlegend=False
    ))
    
    # Add test 95 confidence interval - lower bound
    fig.add_trace(go.Scatter(
        x=test_dates,
        y=ci_lower_95.flatten(),
        mode='lines',
        line=dict(width=0),
        fill='tonexty',
        fillcolor='rgba(255,165,0,0.2)',
        name='95% CI (Test)'
    ))

    # Add test 99 confidence interval - upper bound
    fig.add_trace(go.Scatter(
        x=test_dates,
        y=ci_upper_99.flatten(),
        mode='lines',
        line=dict(width=0),
        showlegend=False
    ))
    
    # Add test 95 confidence interval - lower bound
    fig.add_trace(go.Scatter(
        x=test_dates,
        y=ci_lower_99.flatten(),
        mode='lines',
        line=dict(width=0),
        fill='tonexty',
        fillcolor='rgba(256,165,0,0.2)',
        name='95% CI (Test)'
    ))
    
    # Update historical figure layout
    fig.update_layout(
        title=dict(
            text=f'{wine} <br> LSTM Historical Performance. RMSE: {rmse:.3}',
            x=0.5,  # Center the title horizontally
            xanchor='center'
        ),
        xaxis_title='Date',
        yaxis_title='Price Change',
        legend_title='Legend',
        height=400,
        width=800,
        template='plotly_white',
        hovermode='x unified'
    )
    
    # Create figure for future predictions
    future_fig = go.Figure()
    
    # Add future predictions - only show the selected number of months
    display_month_strings = future_month_strings[:display_months]
    display_predictions = future_predictions[:display_months]
    display_lower_95 = future_lower_95[:display_months]
    display_upper_95 = future_upper_95[:display_months]
    display_lower_99 = future_lower_99[:display_months]
    display_upper_99 = future_upper_99[:display_months]
    
    # Add future predictions line
    future_fig.add_trace(go.Scatter(
        x=display_month_strings, 
        y=display_predictions.flatten(),
        mode='lines+markers',
        name=f'Future Predictions ({horizon})',
        line=dict(color='purple'),
        marker=dict(size=8)
    ))
    
    # Add future 95 confidence interval - upper bound
    future_fig.add_trace(go.Scatter(
        x=display_month_strings,
        y=display_upper_95.flatten(),
        mode='lines',
        line=dict(width=0),
        showlegend=False
    ))
    
    # Add future 95 confidence interval - lower bound
    future_fig.add_trace(go.Scatter(
        x=display_month_strings,
        y=display_lower_95.flatten(),
        mode='lines',
        line=dict(width=0),
        fill='tonexty',
        fillcolor='rgba(128,0,128,0.2)',
        name=f'95% CI ({horizon})'
    ))

    # Add future 99 confidence interval - upper bound
    future_fig.add_trace(go.Scatter(
        x=display_month_strings,
        y=display_upper_99.flatten(),
        mode='lines',
        line=dict(width=0),
        showlegend=False
    ))
    
    # Add future 99 confidence interval - lower bound
    future_fig.add_trace(go.Scatter(
        x=display_month_strings,
        y=display_lower_99.flatten(),
        mode='lines',
        line=dict(width=0),
        fill='tonexty',
        fillcolor='rgba(127,0,128,0.2)',
        name=f'95% CI ({horizon})'
    ))
    
    # Update future figure layout
    future_fig.update_layout(
        title=dict(text=f'Future Predictions ({horizon})',
                   x=0.5,  # Center the title horizontally
                   xanchor='center'),
        xaxis_title='Month',
        yaxis_title='Price Change',
        legend_title='Legend',
        height=400,
        width=800,
        template='plotly_white',
        hovermode='x unified'
    )
    
    # Create VaR figure
    var_fig = go.Figure()
    
    # Calculate Profit/Loss based on future predictions
    pnl_values = [investment * pred for pred in future_predictions[:display_months].flatten()]

    # Add PnL line
    var_fig.add_trace(go.Scatter(
        x=display_month_strings,
        y=pnl_values,
        mode='lines+markers',
        name='Expected PnL',
        line=dict(color='green'),
        marker=dict(size=8)
    ))

    # Calculate historical VaR amount for the current investment
    historical_var_amount_95 = investment * historical_var_95
    historical_var_amount_99 = investment * historical_var_99
    
    # Scale Deep VaR amounts for the current investment
    scaled_deep_var_amounts_95 = [amount * (investment / 1000) for amount in deep_var_amounts_95]
    scaled_deep_var_amounts_99 = [amount * (investment / 1000) for amount in deep_var_amounts_99]
    
    # Assuming future_predictions are percentage changes
    
    # Add HS VaR 95 line
    var_fig.add_trace(go.Scatter(
        x=display_month_strings,
        y=[historical_var_amount_95] * len(display_month_strings),
        mode='lines',
        name='HS VaR 95CI',
        line=dict(color='red', dash='dash')
    ))

    # Add HS VaR 99 line
    var_fig.add_trace(go.Scatter(
        x=display_month_strings,
        y=[historical_var_amount_99] * len(display_month_strings),
        mode='lines',
        name='HS VaR 99CI',
        line=dict(color='orange', dash='dash')
    ))

    # Add Deep VaR 95 line
    var_fig.add_trace(go.Scatter(
        x=display_month_strings,
        y=scaled_deep_var_amounts_95[:display_months],
        mode='lines+markers',
        name='Deep VaR 95CI',
        line=dict(color='blue'),
        marker=dict(size=8)
    ))
    
    # Add Deep VaR 95 line
    var_fig.add_trace(go.Scatter(
        x=display_month_strings,
        y=scaled_deep_var_amounts_99[:display_months],
        mode='lines+markers',
        name='Deep VaR 99CI',
        line=dict(color='purple'),
        marker=dict(size=8)
    ))
  
    # Update VaR figure layout
    var_fig.update_layout(
        title=dict(text=f'Value at Risk and Expected PnL for ${investment} Investment',
                   x=0.5,  # Center the title horizontally
                   xanchor='center'),
        xaxis_title='Month',
        yaxis_title='Amount ($)',
        legend_title='Legend',
        height=400,
        width=800,
        template='plotly_white',
        hovermode='x unified'
    )
    
    # Add a horizontal line at y=0 to better visualize profit vs loss
    var_fig.add_shape(
        type="line",
        x0=display_month_strings[0],
        y0=0,
        x1=display_month_strings[-1],
        y1=0,
        line=dict(color="black", width=1, dash="dot")
    )

    # Set y-axis tick interval to 10 units
    #var_fig.update_yaxes(dtick=10)
    
    # Create a table to show VaR comparison
    var_comparison = pd.DataFrame({
        'Month': future_month_strings[:display_months],
        'Expected PnL ($)': pnl_values,
        'HS VaR 95CI (%)': [historical_var_95] * display_months,
        'HS VaR 95CI ($)': [historical_var_amount_95] * display_months,
        'HS VaR 99CI (%)': [historical_var_99] * display_months,
        'HS VaR 99CI ($)': [historical_var_amount_99] * display_months,
        'Deep VaR 95CI (%)': deep_var_values_95[:display_months],
        'Deep VaR 95CI ($)': scaled_deep_var_amounts_95[:display_months],
        'Deep VaR 99CI (%)': deep_var_values_99[:display_months],
        'Deep VaR 99CI ($)': scaled_deep_var_amounts_99[:display_months],
        
    })
    
    table_fig = go.Figure(data=[go.Table(
        header=dict(values=list(var_comparison.columns),
                    fill_color='paleturquoise',
                    align='left'),
        cells=dict(values=[var_comparison[col] for col in var_comparison.columns],
                  fill_color='lavender',
                  align='left',
                  format=[None, '.4f', '.2f', '.4f', '.2f', '.2f'])
    )])
    
    table_fig.update_layout(
        title=dict(text=f'VaR and PnL Comparison ({horizon})',
                   x=0.5,
                   xanchor='center'),
        height=400,
        width=800
    )
    
    return fig, future_fig, var_fig, table_fig

# Function to handle widget changes
def on_widget_change(change):
    with output:
        clear_output(wait=True)
        selected_wine = wine_dropdown.value
        selected_horizon = forecast_horizon.value
        investment_value = investment_slider.value
        try:
            fig, future_fig, var_fig, table_fig = plot_wine_forecast_and_var(
                selected_wine, selected_horizon, investment_value
            )
            if fig and future_fig and var_fig and table_fig:
                fig.show()
                future_fig.show()
                var_fig.show()
                table_fig.show()
        except Exception as e:
            print(f"Error generating plot: {e}")
            import traceback
            traceback.print_exc()

# Connect the widgets to the handler function
wine_dropdown.observe(on_widget_change, names='value')
forecast_horizon.observe(on_widget_change, names='value')
investment_slider.observe(on_widget_change, names='value')

# Create output area
output = widgets.Output()

# Display the widgets and output area
display(widgets.VBox([
    widgets.HBox([wine_dropdown, forecast_horizon]),
    investment_slider
]))
display(output)

# Initialize with the first wine and 3-month forecast
with output:
    try:
        fig, future_fig, var_fig, table_fig = plot_wine_forecast_and_var(
            wine_dropdown.value, forecast_horizon.value, investment_slider.value
        )
        if fig and future_fig and var_fig and table_fig:
            fig.show()
            future_fig.show()
            var_fig.show()
            table_fig.show()
    except Exception as e:
        print(f"Error generating initial plot: {e}")
        import traceback
        traceback.print_exc()



Output()

## Save model to PC

In [33]:
wine_models

{'Leroy Musigny Grand Cru, Cote de Nuits, France': {'time_series': array([[-3.32019570e-01],
         [-7.36016869e-01],
         [-5.52659529e-01],
         [ 3.57050128e-01],
         [ 7.24250421e-02],
         [-1.63996800e-01],
         [-2.70525927e-01],
         [-3.36802750e-02],
         [-2.14307805e-01],
         [ 3.60339776e-01],
         [-3.77000031e-01],
         [-9.25807301e-01],
         [ 1.12199356e+00],
         [-3.39014000e-01],
         [ 3.11131377e-01],
         [ 8.93304802e-01],
         [-6.84843831e-01],
         [ 5.02468794e-01],
         [ 5.02384799e-01],
         [-8.89090661e-01],
         [-1.00092634e+00],
         [ 1.27852866e+00],
         [-2.21812413e-01],
         [ 5.45798442e-01],
         [-9.81843051e-02],
         [-1.08466429e+00],
         [ 7.00061127e-01],
         [ 9.81120075e-02],
         [-3.75481371e-01],
         [-2.92955580e-01],
         [ 1.89599409e-01],
         [ 1.44213542e+00],
         [ 7.99833891e-01],
         [ 

In [35]:
# Define the directory to save models and scalers
path = 'VScode/LSTM-wine-streamlit-main/'
MODEL_SAVE_DIR = "LSTM-wine-streamlit-main/saved_models"

# Create the directory if it doesn't exist
# Create the directory if it doesn't exist
os.makedirs(MODEL_SAVE_DIR, exist_ok=True)
try:
    sanitized_name = wine.replace(" ", "_").replace("/", "_")
    model_path = os.path.join(MODEL_SAVE_DIR, f"model_{sanitized_name}.keras")
    scaler_path = os.path.join(MODEL_SAVE_DIR, f"scaler_{sanitized_name}.joblib")

    wine_models.save(model_path)
    joblib.dump(scaler, scaler_path)

    print(f"Successfully saved model to: {model_path}")
    print(f"Successfully saved scaler to: {scaler_path}")

except Exception as e:
    print(f"!!! Error saving model or scaler for {wine}: {e}")

print("\n--- Model training and saving process complete. ---")

!!! Error saving model or scaler for Leroy Domaine d'Auvenay Les Gouttes d'Or, Meursault Premier Cru, France: 'dict' object has no attribute 'save'

--- Model training and saving process complete. ---


## Extract comparision table

In [26]:
# Create an empty list to store comparison data for each wine
all_wine_comparisons = []

# Set parameters 
investment = 10000  # Changed from tuple (10,000) to integer 10000
selected_horizon = '12 Months'
display_months = 12

# Loop through all wines
for wine in wine_columns:
    try:
        # Get model data for this wine
        if wine not in wine_models:
            print(f"No model found for {wine}")
            continue
            
        model_data = wine_models[wine]
        
        future_predictions = model_data['future_predictions']
        
        # Get the last date in the dataset
        last_date = df['Datetime'].iloc[-1]
        
        # Generate future month strings
        future_month_strings = generate_future_month_strings(last_date, display_months)
        
        # Calculate values based on investment
        # Convert numpy arrays to float before multiplication
        historical_var_95 = float(model_data['historical_var_95'])
        historical_var_99 = float(model_data['historical_var_99'])
        
        historical_var_amount_95 = investment * historical_var_95
        historical_var_amount_99 = investment * historical_var_99
    
        historical_var_amount_95 = investment * historical_var_95
        historical_var_amount_99 = investment * historical_var_99
    
    # Scale Deep VaR amounts for the current investment
        scaled_deep_var_amounts_95 = [amount * (investment / 1000) for amount in deep_var_amounts_95]
        scaled_deep_var_amounts_99 = [amount * (investment / 1000) for amount in deep_var_amounts_99]
        pnl_values = [investment * pred for pred in future_predictions[:display_months].flatten()]
        
        # Create comparison data for this wine
        # Convert lists to numpy arrays for consistent operations
        pnl_array = np.array(pnl_values)
        deep_var_95_array = np.array(scaled_deep_var_amounts_95[:display_months])
        deep_var_99_array = np.array(scaled_deep_var_amounts_99[:display_months])
        
        var_comparison = pd.DataFrame({
            "Wine_name": wine,
            'Month': future_month_strings[:display_months],
            'Expected PnL ($)': pnl_array,
            'HS VaR 95CI ($)': [historical_var_amount_95] * display_months,
            'HS VaR 95_diff': [historical_var_amount_95 - pnl for pnl in pnl_array],
            'HS VaR 99CI ($)': [historical_var_amount_99] * display_months,
            'HS VaR 99_diff': [historical_var_amount_99 - pnl for pnl in pnl_array],
            'Deep VaR 95CI ($)': deep_var_95_array,
            'Deep VaR 95_diff': deep_var_95_array - pnl_array,
            'Deep VaR 99CI ($)': deep_var_99_array,
            'Deep VaR 99_diff': deep_var_99_array - pnl_array
        })
        
        # Add to the list
        all_wine_comparisons.append(var_comparison)
        
        print(f"Processed {wine}")
        
    except Exception as e:
        print(f"Error processing {wine}: {e}")

# Combine all wine data into a single DataFrame
if all_wine_comparisons:
    combined_comparison = pd.concat(all_wine_comparisons, ignore_index=True)
    
    # Export to CSV
    csv_filename = f"wine_investment_comparison_10k_12months.csv"
    full_path = os.path.join(path, csv_filename)
    combined_comparison.to_csv(full_path, index=False)
    print(f"Exported comparison data to {full_path}")
else:
    print("No wine data was processed successfully")


Processed Leroy Musigny Grand Cru, Cote de Nuits, France
Processed Leroy Domaine d'Auvenay Chevalier-Montrachet Grand Cru, Cote de Beaune, France
Processed Leroy Domaine d'Auvenay Criots-Batard-Montrachet Grand Cru, Cote de Beaune, France
Processed Leroy Chambertin Grand Cru, Cote de Nuits, France
Processed Leroy Domaine d'Auvenay Mazis-Chambertin Grand Cru, Cote de Nuits, France
Processed Leroy Domaine d'Auvenay Les Bonnes-Mares Grand Cru, Cote de Nuits, France
Processed Leroy Corton-Charlemagne Grand Cru, Cote de Beaune, France
Processed Domaine de la Romanee-Conti Montrachet Grand Cru, Cote de Beaune, France
Processed Leroy Richebourg Grand Cru, Cote de Nuits, France
Processed Leroy Romanee-Saint-Vivant Grand Cru, Cote de Nuits, France
Processed Leroy Domaine d'Auvenay Meursault Chaumes des Perrieres, Cote de Beaune, France
Processed Leroy Domaine d'Auvenay Meursault, Cote de Beaune, France
Processed Leroy Domaine d'Auvenay Puligny-Montrachet en La Richarde, Cote de Beaune, France
P

In [28]:
# Create an empty list to store summary data for each wine
all_summaries = []

# Create a comparison DataFrame with mean values for each wine
for wine in wine_columns:
    try:
        # Get model data for this wine
        if wine not in wine_models:
            print(f"No model found for {wine}")
            continue
            
        model_data = wine_models[wine]
        
        # Get the required arrays and values
        future_predictions = model_data['future_predictions']
        historical_var_95 = float(model_data['historical_var_95'])
        historical_var_99 = float(model_data['historical_var_99'])
        deep_var_95_array = np.array(model_data['deep_var_values_95'][:12])  # Get first 12 months
        deep_var_99_array = np.array(model_data['deep_var_values_99'][:12])  # Get first 12 months
        
        # Calculate PnL array
        pnl_array = np.array([investment * pred[0] for pred in future_predictions[:12]])  # Get first 12 months
        
        # Create summary for this wine
        summary_data = {
            "Wine_name": wine,
            'Mean_Expected_PnL': pnl_array.mean(),
            'Mean_HS_VaR_95': historical_var_95 * investment,
            'Mean_HS_VaR_95_diff': np.mean([historical_var_95 * investment - pnl for pnl in pnl_array]),
            'Mean_HS_VaR_99': historical_var_99 * investment,
            'Mean_HS_VaR_99_diff': np.mean([historical_var_99 * investment - pnl for pnl in pnl_array]),
            'Mean_Deep_VaR_95': np.mean(deep_var_95_array),
            'Mean_Deep_VaR_95_diff': np.mean(deep_var_95_array - pnl_array),
            'Mean_Deep_VaR_99': np.mean(deep_var_99_array),
            'Mean_Deep_VaR_99_diff': np.mean(deep_var_99_array - pnl_array)
        }
        
        all_summaries.append(summary_data)
        print(f"Processed summary for {wine}")
        
    except Exception as e:
        print(f"Error processing summary for {wine}: {e}")

# Create the final summary DataFrame
summary_comparison = pd.DataFrame(all_summaries)

# Export summary to CSV
summary_csv_filename = f"wine_investment_summary_10k_12months_mean.csv"
summary_full_path = os.path.join(path, summary_csv_filename)
summary_comparison.to_csv(summary_full_path, index=False)
print(f"Exported summary data to {summary_full_path}")

# Display the summary DataFrame
display(summary_comparison)

Processed summary for Leroy Musigny Grand Cru, Cote de Nuits, France
Processed summary for Leroy Domaine d'Auvenay Chevalier-Montrachet Grand Cru, Cote de Beaune, France
Processed summary for Leroy Domaine d'Auvenay Criots-Batard-Montrachet Grand Cru, Cote de Beaune, France
Processed summary for Leroy Chambertin Grand Cru, Cote de Nuits, France
Processed summary for Leroy Domaine d'Auvenay Mazis-Chambertin Grand Cru, Cote de Nuits, France
Processed summary for Leroy Domaine d'Auvenay Les Bonnes-Mares Grand Cru, Cote de Nuits, France
Processed summary for Leroy Corton-Charlemagne Grand Cru, Cote de Beaune, France
Processed summary for Domaine de la Romanee-Conti Montrachet Grand Cru, Cote de Beaune, France
Processed summary for Leroy Richebourg Grand Cru, Cote de Nuits, France
Processed summary for Leroy Romanee-Saint-Vivant Grand Cru, Cote de Nuits, France
Processed summary for Leroy Domaine d'Auvenay Meursault Chaumes des Perrieres, Cote de Beaune, France
Processed summary for Leroy D

,Wine_name,Mean_Expected_PnL,Mean_HS_VaR_95,Mean_HS_VaR_95_diff,Mean_HS_VaR_99,Mean_HS_VaR_99_diff,Mean_Deep_VaR_95,Mean_Deep_VaR_95_diff,Mean_Deep_VaR_99,Mean_Deep_VaR_99_diff
0,"Leroy Musigny Grand Cru, Cote de Nuits, France",148.599106,1551.0,1402.400879,1908.0,1759.400879,0.007837,-148.591264,0.043871,-148.555230
1,Leroy Domaine d'Auvenay Chevalier-Montrachet G...,249.381302,770.0,520.618713,1380.0,1130.618652,-0.024887,-249.406204,-0.024951,-249.406268
2,Leroy Domaine d'Auvenay Criots-Batard-Montrach...,351.748077,1326.0,974.251770,3449.0,3097.251953,-0.034500,-351.782598,-0.035482,-351.783579
3,"Leroy Chambertin Grand Cru, Cote de Nuits, France",170.454926,792.0,621.545044,1209.0,1038.545166,-0.016921,-170.471848,-0.016667,-170.471594
4,Leroy Domaine d'Auvenay Mazis-Chambertin Grand...,269.610443,639.0,369.389557,1451.0,1181.389648,-0.026255,-269.636653,-0.024832,-269.635229
5,Leroy Domaine d'Auvenay Les Bonnes-Mares Grand...,194.403275,521.0,326.596680,743.0,548.596741,-0.019271,-194.422560,-0.019502,-194.422791
6,"Leroy Corton-Charlemagne Grand Cru, Cote de Be...",240.427246,906.0,665.572693,1687.0,1446.572754,-0.023843,-240.451077,-0.023408,-240.450642
7,Domaine de la Romanee-Conti Montrachet Grand C...,100.280617,321.0,220.719406,445.0,344.719360,-0.009712,-100.290322,-0.010148,-100.290758
8,"Leroy Richebourg Grand Cru, Cote de Nuits, France",146.537125,349.0,202.462891,522.0,375.462891,-0.014614,-146.551719,-0.014614,-146.551719
9,"Leroy Romanee-Saint-Vivant Grand Cru, Cote de ...",815.373535,483.0,-332.373535,753.0,-62.373547,-0.081370,-815.454916,-0.080988,-815.454533
